## Pricing Function

The [Heston process](https://en.wikipedia.org/wiki/Heston_model) is described by the SDE: 

$ \begin{cases}
dS_t = \mu S_t dt + \sqrt{v_t} S_t dW^1_t \\
dv_t = \kappa (\theta - v_t) dt + \sigma \sqrt{v_t} dW^2_t 
\end{cases}$

The stock price follows a "geometric Brownian motion" with a stochastic volatility. The square of the volatility (the variance) follows a CIR process.     

Parameters from the market are:
- $\mu$ drift of the stock process
- $S_t$ price of the underlying asset

Parameters to be calibrated are:
- $v_t$ the instantaneous variance from the second equation 
- $\kappa$ mean reversion coefficient of the variance process
- $\theta$ long term mean of the variance process 
- $\sigma$  volatility coefficient of the variance process
- $\rho$ correlation between $W^1$ and $W^2$ i.e.
$$ dW^1_t dW^2_t = \rho dt $$

We will also require that $2\kappa \theta > \sigma^2$ (Feller condition).


$
C(K,T) = \frac{\operatorname{exp}(-\alpha \operatorname{log}(K))}{\pi}
\int^{+\infty}_{0} \operatorname{exp}(-iv\operatorname{log}(K))\xi(v)dv
$

where

$
\xi(v) = \frac{\operatorname{exp}(-rT)E[\operatorname{exp}(i(v-(\alpha+1)i) \operatorname{log}(S_T))]}{\alpha^2 + \alpha - v^2 + i(2\alpha + 1)v}\\
\ \ \ \  \; = \frac{\operatorname{exp}(-rT)\phi(v-(\alpha+1)i,T)}
{\alpha^2 + \alpha - v^2 + i(2\alpha + 1)v}
$

## Heston characteristic function as proposed by Schoutens (2004)

$$
\phi(u,t) = E[exp(iu \operatorname{log}(S_t))|S_0, v_0] \\
\qquad \ = \operatorname{exp}(iu(\operatorname{log}S_0 + (r-q)t)) \\
\qquad \;  \times \operatorname{exp}(\theta\kappa\sigma^{-2}((\kappa - \rho \sigma u i -d)t - 2 \operatorname{log}((1-g \operatorname{e}^{-dt})/(1-g)))) \\
\qquad \; \times \operatorname{exp}(v_0 \sigma^{-2}(\kappa - \rho \sigma u i -d)(1-\operatorname{e}^{-dt})/(1-g\operatorname{e}^{-dt}))
$$

where
$$
d = ((\rho \sigma u i - \kappa)^2 - \theta^2(-iu-u^2))^{\frac{1}{2}} \\
g = (\kappa - \rho \sigma u i - d) / (\kappa - \rho \sigma u i + d)
$$

notes: $\mu=r-q$ in the codes below

In [ ]:
# CF
def cf_Heston_schoutens(u, S, Tau, r, v0, kappa, theta, rho, vol_vol):
    """
    Heston characteristic function as proposed by Schoutens (2004)
    S      : Spot Price
    Tau    : Time to Maturity
    r      : Risk-Free Rate
    v0     : Instantaneous vol
    kappa  : Mean reversion rate
    theta  : Long term vol
    rho    : Corr of two Brownian Motion
    vol_vol: Vol of vol
    """
    phi = kappa - vol_vol * rho * u * 1j
    d = np.sqrt(((-phi) ** 2) - ((vol_vol ** 2) * (-1j * u - u ** 2)))
    g = (phi - d) / (phi + d)

    cf = np.exp((1j * u) * (np.log(S) + r * Tau)) * \
         np.exp((kappa * theta) / (vol_vol ** 2) * ((phi - d) * Tau - 2 * np.log((1 - g * np.exp(-d * Tau)) / (1 - g)))) * \
         np.exp((v0 / vol_vol ** 2) * (phi - d) * (1 - np.exp(-d * Tau)) / (1 - g * np.exp(-d * Tau)))
    return cf

In [ ]:
# Xi in pricing function
def xi(u, alpha, S, Tau, r, v0, kappa, theta, rho, vol_vol):
    """
    Intermediate step of calculating the call price
    alpha : alpha th moment
    """
    cf = cf_Heston_schoutens(u - (alpha+1)*1j,
                             S, Tau, r, v0, kappa, theta, rho, vol_vol)
    numerator = np.exp(-r * Tau)*cf
    denominator = alpha**2 + alpha - u**2 + 1j*(2*alpha+1)*u
    return numerator/denominator


In [ ]:
# Vectorised integrand 
def integrand(u, alpha, S, K, Tau, r, v0, kappa, theta, rho, vol_vol):
    return (np.exp(-1j*u*np.log(K)))*xi(u, alpha, S, Tau, r, v0, kappa, theta, rho, vol_vol)


def quad_integrand(alpha, S, K, Tau, r, v0, kappa, theta, rho, vol_vol):
    return quad(integrand, limit=10000, a=0, b=np.inf, args=(alpha, S, K, Tau, r, v0, kappa, theta, rho, vol_vol))[0]


vec_quad_integrand = np.vectorize(quad_integrand)

In [ ]:
# Pricing Function
def call_price_HS(alpha, S, K, Tau, r, par):
    v0, kappa, theta, rho, vol_vol = par
    multiplier = np.exp(-alpha*np.log(K))/np.pi
    return multiplier*vec_quad_integrand(alpha, S, K, Tau, r, v0, kappa, theta, rho, vol_vol)